In [1]:
import argparse
import json
import os
import gc
import torch

import sys
sys.path.append("./")

os.environ['CUDA_VISIBLE_DEVICES']='1,3'
device = 'cuda'

import peft
from peft import PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoConfig
from huggingface_hub import hf_hub_download


model = LlamaForCausalLM.from_pretrained('model/chinese-llama-alpaca-plus-lora-7b/', device_map='auto')
# base_model = LlamaForCausalLM.from_pretrained('decapoda-research/llama-7b-hf', device_map='auto')
# model = PeftModel.from_pretrained(
#     base_model,
#     'ziqingyang/chinese-llama-plus-lora-7b',
#     device_map='auto',
#     torch_dtype=torch.float16
# )

/home/lzw/miniconda3/envs/Bert/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:24<00:00, 12.06s/it]


In [3]:
model_path = "./model/chinese-llama-alpaca-plus-lora-7b"
config = AutoConfig.from_pretrained(
    model_path,
)
with torch.no_grad():
    torch_dtype = torch.float16
    model = LlamaForCausalLM.from_pretrained(
        model_path,
        config=config,
        torch_dtype=torch_dtype,
        # low_cpu_mem_usage=True
    )
    model = model.cuda()

    tokenizer = LlamaTokenizer.from_pretrained(
        model_path,
    )
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model.resize_token_embeddings(len(tokenizer))

    # 加载lora
    peft_model_path = "./model/ChatMed-Consult_llama_lora_pt_v0"
    model = PeftModel.from_pretrained(model, peft_model_path)
    model.eval()


Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.40s/it]


In [4]:
generation_config = dict(
    temperature=0.2,
    # top_k=40,
    top_p=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.3,
    max_new_tokens=400
)

while True:
    device = torch.device('cuda')
    inputs = tokenizer(input(''), return_tensors="pt", add_special_tokens=False)
    output = tokenizer(
        inputs.input_ids.to(device),
        inputs.attention_mask.to(device),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        **generation_config
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(output)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 12>:14                                                                            │
│                                                                                                  │
│   11 while True:                                                                                 │
│   12 │   device = torch.device('cuda')                                                           │
│   13 │   inputs = tokenizer(input(''), return_tensors="pt", add_special_tokens=False)            │
│ ❱ 14 │   output = tokenizer(                                                                     │
│   15 │   │   inputs.input_ids.to(device),                                                        │
│   16 │   │   inputs.attention_mask.to(device),                                                   │
│   17 │   │   eos_token_id=tokenizer.eos_token_id,                                                │
│                                                                                                  │
│ /home/lzw/miniconda3/envs/Bert/lib/python3.8/site-packages/transformers/tokenization_utils_base. │
│ py:2548 in __call__                                                                              │
│                                                                                                  │
│   2545 │   │   │   # input mode in this case.                                                    │
│   2546 │   │   │   if not self._in_target_context_manager:                                       │
│   2547 │   │   │   │   self._switch_to_input_mode()                                              │
│ ❱ 2548 │   │   │   encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)      │
│   2549 │   │   if text_target is not None:                                                       │
│   2550 │   │   │   self._switch_to_target_mode()                                                 │
│   2551 │   │   │   target_encodings = self._call_one(text=text_target, text_pair=text_pair_targ  │
│                                                                                                  │
│ /home/lzw/miniconda3/envs/Bert/lib/python3.8/site-packages/transformers/tokenization_utils_base. │
│ py:2606 in _call_one                                                                             │
│                                                                                                  │
│   2603 │   │   │   │   return False                                                              │
│   2604 │   │                                                                                     │
│   2605 │   │   if not _is_valid_text_input(text):                                                │
│ ❱ 2606 │   │   │   raise ValueError(                                                             │
│   2607 │   │   │   │   "text input must of type `str` (single example), `List[str]` (batch or s  │
│   2608 │   │   │   │   "or `List[List[str]]` (batch of pretokenized examples)."                  │
│   2609 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or 
`List[List[str]]` (batch of pretokenized examples).